Ноутбук с парсингом новостных статей

* 0. Инициализация
* 1. New york times
* 2. The Guardian

In [11]:
from utils_create_news import try_load, save_s3
from datetime import datetime, timedelta
from dotenv import load_dotenv
from minio import Minio

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import os
import re
import time
from bs4 import BeautifulSoup
import requests

# 0. Инициализация

In [4]:
load_dotenv()

YANDEX_CLOUD_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID_2")
YANDEX_CLOUD_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY_2")

API_KEY_NYT = os.getenv("API_KEY_NYT")
API_KEY_THE_GUARDIAN = os.getenv("API_KEY_THE_GUARDIAN")

BUCKET_NAME = 'graduate' # s3

PATH = '/Users/dan/git_repo/graduate/'

In [5]:
client_s3 = Minio(
    "storage.yandexcloud.net",
    access_key=YANDEX_CLOUD_ACCESS_KEY,
    secret_key=YANDEX_CLOUD_SECRET_KEY,
    secure=True
)

# 1. New york times

Сначала по API получим URL + Название статьи, затем спарсим текст

In [ ]:
# Выгружаем URL и Titile
# news_dict_nyt_url = dict()
news_dict_nyt_url = try_load(file_path=PATH+'data/', file_name='news_dict_nyt_url.pkl', client_s3=client_s3)

# Параметры запроса
year_list = [2025,2025,2025,2024,2024,2024,2024,2024]
month_list = [3,2,1,12,11,10,8,7]
category = "Politics"

for step in range(len(month_list)):

    year = year_list[step]
    month = month_list[step]
    # Формируем URL запроса
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={API_KEY_NYT}"

    # Запрос к API NYT
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        
        # Извлекаем все статьи
        all_articles = data['response']['docs']

        for i in range(len(all_articles)):
            if all_articles[i]['news_desk'] == 'Politics':
                titile = all_articles[i]['headline']['main']
                url = all_articles[i]['web_url']

                news_dict_nyt_url[titile] = url
        
    else:
        print(f"Ошибка запроса: {response.status_code}")
        print(response.text)

In [ ]:
# Спарсим сами статьи
papers_nyt = try_load(file_path=PATH+'data/', file_name='papers_nyt.pkl', client_s3=client_s3)

news_dict_keys = list(news_dict_nyt_url.keys())

# налету уберем блоки, не относящиеся к статье
pattern = re.compile(r'^(Advertisement|Supported by|By \w+ \w+|Reporting from \w+|Fact Check|Trump Administration)$')

# Создаем сессию
session = requests.Session()

# Обновляем заголовки сессии
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.64',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0',
    'TE': 'Trailers',
    'Dnt': '1', 
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Pragma': 'no-cache'
})

for i, title in enumerate(news_dict_keys):
    if i%25==0:
        print(i)
        with open('papers_nyt.pkl', 'wb') as file:
            pickle.dump(papers_nyt, file)

    url = news_dict_nyt_url[title]

    if title not in papers_nyt.keys():
        time.sleep(1)
        # GET-запрос
        response = session.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Попробуем найти основной текст статьи
            # Вариант 1: ищем по <article>
            article_body = soup.find('article')

            # Вариант 2: ищем по div с более общими классами
            if not article_body:
                article_body = soup.find('div', {'class': 'css-1v6mj2p'})

            # Если не нашли, пробуем другие элементы
            if not article_body:
                article_body = soup.find('section')

            if article_body:
                paragraphs = article_body.find_all('p')

                full_text = '\n'.join(p.get_text() for p in paragraphs)
                
                # Небольшая предобработка
                full_text = " ".join(line for line in full_text.splitlines() if not pattern.match(line))
                full_text = full_text.split('We are having trouble retrieving the article content.')[0]

                papers_nyt[title] = full_text

            else:
                print("Не удалось найти текст статьи на странице.")
        else:
            print()
            print(f"Ошибка при запросе: {response.status_code}, {url}")
            break

In [ ]:
# Сохраним спаршенные данные
with open(PATH+'data/papers_nyt.pkl', 'wb') as file:
    pickle.dump(papers_nyt, file)

save_s3(pickle_data=papers_nyt, object_key='papers_nyt.pkl', client_s3=client_s3)

# 2. The Guardian

In [47]:
# Загрузим оригинальные новости
paper_dict = try_load(file_path=PATH+'data/', file_name='the_guardian_politic_CLEAN_news_09022025.pkl', client_s3=client_s3)

query = "politics"  
from_date = (datetime.now() - timedelta(days=60)).strftime('%Y-%m-%d')
to_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
page_size = 200  

# Формируем URL запроса
url = f"https://content.guardianapis.com/search?q={query}&from-date={from_date}&to-date={to_date}&page-size={page_size}&show-fields=body,headline,trailText,byline&show-tags=keyword&api-key={API_KEY_THE_GUARDIAN}"

# Делаем запрос к API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    articles = data['response']['results']
    
    for article in articles:
        # Проверяем, что статья относится к политике
        if any(tag['webTitle'] == 'Politics' for tag in article.get('tags', [])):
            
            url = article['id']
            titile = article['fields']['headline']
            paper = article['fields']['body']

            paper_dict[url] = [titile, paper]
    
else:
    print(f"Ошибка запроса: {response.status_code}")
    print(response.text)

len(paper_dict)

5200

In [ ]:
# Сохраним спаршенные данные
with open(PATH+'data/the_guardian_politic_CLEAN_news_09022025.pkl', 'wb') as file:
    pickle.dump(paper_dict, file)

save_s3(pickle_data=paper_dict, object_key='the_guardian_politic_CLEAN_news_09022025.pkl', client_s3=client_s3)

#